In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [3]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1,28,28,1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [4]:
# Layer 1
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01), name='filter1')
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding='SAME', name='conv1')
L1 = tf.nn.relu(L1, name='acti1')
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

In [5]:
# Layer 2
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01), name='filter2')
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding='SAME', name='conv2')
L2 = tf.nn.relu(L2, name='acti2')
L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME', name='pool2')
L2 = tf.nn.dropout(L2, keep_prob = keep_prob)

In [6]:
# Layer 3
W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01), name='filter3')
L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding='SAME', name='conv3')
L3 = tf.nn.relu(L3, name='acti3')
L3 = tf.nn.max_pool(L3, ksize = [1,2,2,1], strides = [1,2,2,1], padding='SAME', name='pool2')
L3 = tf.nn.dropout(L3, keep_prob = keep_prob)

In [7]:
# Layer 4 FC
L4 = tf.reshape(L3, [-1, 128 * 4 * 4])
W4 = tf.get_variable('W4', shape = [128 * 4 * 4, 625], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
Z4 = tf.matmul(L4, W4) + b4
A4 = tf.nn.relu(Z4)
L4 = tf.nn.dropout(A4, keep_prob = keep_prob)

In [8]:
# Layer 5 FC
W5 = tf.get_variable('W5', shape = [625, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

In [9]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
training_epochs = 20
batch_size = 100
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train model
print('learning started. It takes some time')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning Finished !')

learning started. It takes some time
Epoch: 0001 cost = 0.593505287
Epoch: 0002 cost = 0.326109601
Epoch: 0003 cost = 0.309570075
Epoch: 0004 cost = 0.294031893
Epoch: 0005 cost = 0.294958035
Epoch: 0006 cost = 0.283965291
Epoch: 0007 cost = 0.296587170
Epoch: 0008 cost = 0.284776956
Epoch: 0009 cost = 0.282700342
Epoch: 0010 cost = 0.288448098
Epoch: 0011 cost = 0.285927345
Epoch: 0012 cost = 0.288150462
Epoch: 0013 cost = 0.296068514
Epoch: 0014 cost = 0.294806283
Epoch: 0015 cost = 0.285564879
Epoch: 0016 cost = 0.290437337
Epoch: 0017 cost = 0.293402086
Epoch: 0018 cost = 0.282151869
Epoch: 0019 cost = 0.276318175
Epoch: 0020 cost = 0.293829412
Learning Finished !


In [15]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
train_accuracy = sess.run(accuracy, feed_dict={X: train_data, Y: train_labels, keep_prob:1})
test_accuracy = sess.run(accuracy, feed_dict={X: test_data, Y: test_labels, keep_prob:1})
print('Train Accuracy', train_accuracy)
print('Test Accuracy', test_accuracy)

Train Accuracy 0.9572545
Test Accuracy 0.9585
